# Ridge Regression Using UCI Dataset


In [57]:
import pandas as pd
import numpy as np

# Communities and Crime dataset for regression
#https://archive.ics.uci.edu/ml/datasets/Communities+and+Crime+Unnormalized

crime_datafile = 'E:\Protfolio\Ridge Regression Using UCI Dataset/CommViolPredUnnormalizedData.txt'
crime = pd.read_table(crime_datafile, sep=',', na_values='?')
crime.head()

,communityname,state,countyCode,communityCode,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,...,burglaries,burglPerPop,larcenies,larcPerPop,autoTheft,autoTheftPerPop,arsons,arsonsPerPop,ViolentCrimesPerPop,nonViolPerPop
0,BerkeleyHeightstownship,NJ,39.0,5320.0,1,11980,3.10,1.37,91.78,6.50,...,14.0,114.85,138.0,1132.08,16.0,131.26,2.0,16.41,41.02,1394.59
1,Marpletownship,PA,45.0,47616.0,1,23123,2.82,0.80,95.57,3.44,...,57.0,242.37,376.0,1598.78,26.0,110.55,1.0,4.25,127.56,1955.95
2,Tigardcity,OR,NaN,NaN,1,29344,2.43,0.74,94.33,3.43,...,274.0,758.14,1797.0,4972.19,136.0,376.30,22.0,60.87,218.59,6167.51
3,Gloversvillecity,NY,35.0,29443.0,1,16656,2.40,1.70,97.35,0.50,...,225.0,1301.78,716.0,4142.56,47.0,271.93,NaN,NaN,306.64,NaN
4,Bemidjicity,MN,7.0,5068.0,1,11245,2.76,0.53,89.16,1.17,...,91.0,728.93,1060.0,8490.87,91.0,728.93,5.0,40.05,NaN,9988.79


In [58]:
# remove features with poor coverage or lower relevance, and keep ViolentCrimesPerPop target column
columns_to_keep = ([5, 6] + list(range(11,26))+ list(range(32, 103)) + [145])
crime = (crime[crime.columns[columns_to_keep]].dropna().reset_index(drop=True))
crime.head()

,population,householdsize,agePct12t21,agePct12t29,agePct16t24,agePct65up,numbUrban,pctUrban,medIncome,pctWWage,...,MedOwnCostPctInc,MedOwnCostPctIncNoMtg,NumInShelters,NumStreet,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,ViolentCrimesPerPop
0,11980,3.10,12.47,21.44,10.93,11.33,11980,100.0,75122,89.24,...,21.1,14.0,11,0,10.66,53.72,65.29,78.09,89.14,41.02
1,23123,2.82,11.01,21.30,10.48,17.18,23123,100.0,47917,78.99,...,20.7,12.5,0,0,8.30,77.17,71.27,90.22,96.12,127.56
2,29344,2.43,11.36,25.88,11.01,10.28,29344,100.0,35669,82.00,...,21.7,11.6,16,0,5.00,44.77,36.60,61.26,82.85,218.59
3,16656,2.40,12.55,25.20,12.19,17.57,0,0.0,20580,68.15,...,20.6,14.5,0,0,2.04,88.71,56.70,90.17,96.24,306.64
4,140494,2.45,18.09,32.89,20.04,13.26,140494,100.0,21577,75.78,...,17.3,11.7,327,4,1.49,64.35,42.29,70.61,85.66,442.95


In [59]:
#splitting features
X=crime.iloc[:,0:88]
X

,population,householdsize,agePct12t21,agePct12t29,agePct16t24,agePct65up,numbUrban,pctUrban,medIncome,pctWWage,...,MedRentPctHousInc,MedOwnCostPctInc,MedOwnCostPctIncNoMtg,NumInShelters,NumStreet,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85
0,11980,3.10,12.47,21.44,10.93,11.33,11980,100.00,75122,89.24,...,23.8,21.1,14.0,11,0,10.66,53.72,65.29,78.09,89.14
1,23123,2.82,11.01,21.30,10.48,17.18,23123,100.00,47917,78.99,...,27.6,20.7,12.5,0,0,8.30,77.17,71.27,90.22,96.12
2,29344,2.43,11.36,25.88,11.01,10.28,29344,100.00,35669,82.00,...,24.1,21.7,11.6,16,0,5.00,44.77,36.60,61.26,82.85
3,16656,2.40,12.55,25.20,12.19,17.57,0,0.00,20580,68.15,...,28.7,20.6,14.5,0,0,2.04,88.71,56.70,90.17,96.24
4,140494,2.45,18.09,32.89,20.04,13.26,140494,100.00,21577,75.78,...,26.4,17.3,11.7,327,4,1.49,64.35,42.29,70.61,85.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,56216,3.07,15.46,30.16,14.34,8.08,56216,100.00,24727,75.05,...,29.8,22.6,11.7,64,0,18.90,52.67,39.19,74.58,85.88
1990,12251,2.68,17.36,31.23,16.97,12.57,12251,100.00,20321,75.06,...,23.8,17.3,14.4,0,0,2.24,75.16,49.12,78.79,92.85
1991,32824,2.46,11.81,20.96,9.53,20.73,32824,100.00,27182,59.79,...,30.5,23.9,13.1,44,0,7.35,48.66,46.73,75.54,92.30
1992,13547,2.89,17.16,30.01,14.73,10.42,0,0.00,19899,71.67,...,26.2,23.3,14.1,0,0,2.28,82.26,54.05,79.72,94.06


In [60]:
#splitting level
y=crime['ViolentCrimesPerPop']
y

0        41.02
1       127.56
2       218.59
3       306.64
4       442.95
         ...  
1989    545.75
1990    124.10
1991    353.83
1992    691.17
1993    918.89
Name: ViolentCrimesPerPop, Length: 1994, dtype: float64

In [61]:
#plitting train and test set
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0)

In [62]:
#fit rdge regreesion on train set
linridge = Ridge(alpha = 200.0)
linridge.fit(X_train, y_train)

Ridge(alpha=200.0)

In [63]:
print('Crime dataset')
print('ridge regression linear model intercept: {}'
     .format(linridge.intercept_))
print('ridge regression linear model coeff:\n{}'
     .format(linridge.coef_))
print('R-squared score (training): {:.3f}'
     .format(linridge.score(X_train, y_train)))
print('R-squared score (test): {:.3f}'
     .format(linridge.score(X_test, y_test)))
print('Number of non-zero features: {}'
     .format(np.sum(linridge.coef_ != 0)))

Crime dataset
ridge regression linear model intercept: -3825.779337666189
ridge regression linear model coeff:
[ 1.94525084e-03  3.60672324e+00  9.77597457e+00 -2.89878070e+01
  1.56491973e+00 -1.61794931e+01 -2.80483384e-03  1.61476288e+00
 -4.49866909e-03 -6.36280925e+00  1.12184492e+01 -5.30290292e+00
  7.85876492e+00  9.07921248e-01 -7.50830688e+00  6.50432612e-03
 -1.47782266e-03  4.58331200e-03 -4.97232323e+00 -1.80237793e+01
  8.99804172e+00 -1.39525804e+00 -2.21934676e-02  9.40602934e+00
 -3.84132075e+00 -3.48794186e+00  4.05684701e+00  7.52380181e+00
  1.26475337e+01  1.14790740e+01 -1.33834365e+01 -7.67055292e+00
  2.94082718e+00  1.40020152e+00 -2.18931681e+01  1.77011396e+00
  2.96422709e-01  3.90848283e+00 -1.12222493e+01 -3.59724737e-03
  4.11061475e+01 -1.23576657e-03  1.21901107e+00  2.91275235e-01
 -2.35878188e+00  1.00126536e+00 -8.90753777e+00 -7.24070266e+00
  3.40650253e+00 -2.33091913e+00  1.22570942e-01 -3.15217545e+01
  4.20196899e+00 -1.43097396e+01  4.63350610